In [ ]:
import csv
import transformers
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, TextStreamer
import torch
import requests
import sentencepiece as spm
import time
import json
import torch
import ast
import openai
from openai import OpenAI

from guidance import models, select, gen

torch.manual_seed(0)


global model
global modelType
## In case a local model from Huggingface is being used the model Type is "Transformers". In case a cloud-based model from DeepInfra is use, the model Type is "DeepInfra"
modelType="Transformers"
model="TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ"
#modelType="DeepInfra"
#model="meta-llama/Meta-Llama-3.1-70B-Instruct"
#model="Qwen/Qwen2.5-72B-Instruct"
#model="mistralai/Mixtral-8x7B-Instruct-v0.1"

## apiKey is the key needed to call DeepInfra
global apiKey
apiKey=""
global topics 
topics = []


if modelType=="Transformers":
    ModelGuidance = models.Transformers(model, device_map='cuda', torch_dtype=torch.bfloat16, echo=False, trust_remote_code=True)
elif modelType=="DeepInfra":
    client = OpenAI(api_key=apiKey,base_url="https://api.deepinfra.com/v1/openai")



In [ ]:
def setParameters(newTemperature):
    global answerTemperature, selectOptions, topics
    answerTemperature = newTemperature
    selectOptions = []

    for topic_data in topics:
        tmpSelectOptions="["
        for category_input, _ in topic_data['categories']:
            tmpSelectOptions=tmpSelectOptions+"'"+category_input.value+"',"
        tmpSelectOptions = tmpSelectOptions[:-1]
        tmpSelectOptions = tmpSelectOptions+"]"
        selectOptions.append(tmpSelectOptions)
        
def getAnswer(prompt, topicIndex, constrainedOutput):
    if modelType=="DeepInfra":
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            max_tokens=15,
            temperature=answerTemperature,
        )
        generated_answer = completion.choices[0].message.content

        print(generated_answer)
        for option in ast.literal_eval(selectOptions[topicIndex]):
            escaped_option = re.escape(option)

            if re.search(escaped_option, generated_answer, re.IGNORECASE):
                ret = option  # Return the matching option
                break
        else:
            ret = "undefined"
        return ret
        
    elif modelType=="Transformers":
        if constrainedOutput==True:
            output=ModelGuidance+f' '+prompt+select(options=ast.literal_eval(selectOptions[topicIndex]),name='answer')
            ret=output["answer"]   
        else:
            output=ModelGuidance+f' '+prompt+gen(max_tokens=15,name='answer')
            generated_answer = output["answer"]
            print(generated_answer)
            for option in ast.literal_eval(selectOptions[topicIndex]):
                escaped_option = re.escape(option)

                if re.search(escaped_option, generated_answer, re.IGNORECASE):
                    ret = option  # Return the matching option
                    break
            else:
                ret = "undefined"


        return ret
    
    
def singleClassification(text, newTemperature, isItASingleClassification, constrainedOutput):
    setParameters(newTemperature)
    ret=[]


    for l in range(len(selectOptions)):
        prompt=topics[l]['prompt'].value
        prompt=prompt.replace('[TOPIC]', topics[l]['topic_input'].value)
        prompt=prompt.replace('[CATEGORIES]', selectOptions[l])
        prompt=prompt.replace('[TEXT]', text)
        answer = getAnswer(prompt,l,constrainedOutput)
        ret.append(answer)
        
        
        if isItASingleClassification:
            print(topics[l]['topic_input'].value+":"+answer)


    return ret


In [ ]:
def groupClassification(dataset,withEvaluation,numberOfCorrectResults, constrainedOutput):
    if os.path.exists(dataset+".csv")==False:
        print("No "+dataset+" file found")
        return
        
        
    startcount=1
    saveName=dataset+"_(result)"
    
    
    with open(dataset+".csv", 'r', encoding='ISO-8859-1') as file:
        reader = csv.reader(file, delimiter=';')
        readerlist = list(reader)
        
        count = -1 
        resultRow=[]
        
        for row in readerlist:

            count+=1              
            if count==0: #write StartInfo/Header into ResultFile
                singleResult=[]
                singleResult.append("")
                elementcounter=-1
                for element in row:
                    elementcounter=elementcounter+1
                    if elementcounter==0:
                        singleResult.append(element)
                    else:
                        numberOfCorrectResults.append(0)
                        if withEvaluation:
                            singleResult.append(element+"(GroundTruth)")
                        singleResult.append(element)
                with open(saveName+".csv",'a',newline='',encoding='utf-8') as f:
                    writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
                    writer.writerow(singleResult)    
                
            if count>=startcount:
                if count!=0:
                    
                    result=singleClassification(row[0], 0.0, False, constrainedOutput)
                    print("Count:"+str(count))              
    
                    singleResult=[]
                    singleResult.append(str(count))
                    singleResult.append(row[0])
                    tmpCount=0
                    for ret in result:
                        tmpCount=tmpCount+1
                        if(withEvaluation==True):
                            if tmpCount < len(row):
                                singleResult.append(row[tmpCount])
                                singleResult.append(ret)
                                if ret==row[tmpCount]:
                                    numberOfCorrectResults[tmpCount-1]+=1
                            else:
                                singleResult.append("UNDEFINED")
                        else:
                            singleResult.append(ret)
                            



                with open(saveName+".csv",'a',newline='',encoding='utf-8') as f:
                    writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
                    writer.writerow(singleResult)


                    resultRow.append(singleResult)

    

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import os

def on_value_change(change):
    if change['new']:  # Check if the new value is True (checkbox checked)
        freeText_input.layout.visibility = 'hidden'
        classify_CSV_box.layout.visibility = 'visible'
    else:
        freeText_input.layout.visibility = 'visible'
        classify_CSV_box.layout.visibility = 'hidden'

def add_topic(b=None, topic_data=None):
    print(freeText_container.children[0].value)
    topic_input = widgets.Text(
        value=topic_data['topic_input'] if topic_data else '',
        description='The topic is:',
        disabled=False
    )
    add_category_button = widgets.Button(description="Add Category")
    remove_topic_button = widgets.Button(description="Remove Topic")
    categories_container = widgets.VBox()

    if topic_data and 'prompt' in topic_data:
        tmpPrompt=topic_data['prompt']
    else:
        tmpPrompt="INSTRUCTION: You are a helpful classifier. You select the correct of the possible categories for classifying a piece of text. The topic of the classification is '[TOPIC]'. The allowed categories are '[CATEGORIES]'. QUESTION: The text to be classified is '[TEXT]'. ANSWER: The correct category for this text is '"

    prompt_input = widgets.Text(
        value=tmpPrompt,
        description='Prompt:',
        disabled=False
    )

    topic_box = widgets.VBox([topic_input, widgets.HBox([prompt_input,add_category_button, remove_topic_button]), categories_container])

    topic_info = {
        'topic_input': topic_input,
        'categories': [],
        'prompt': prompt_input,
        'categories_container': categories_container,
        'topic_box': topic_box
    }





    add_category_button.on_click(lambda btn: add_category(topic_info))
    remove_topic_button.on_click(lambda btn: remove_topic(topic_info))

    topics.append(topic_info)
    update_topics_container()


    if topic_data and 'categories' in topic_data:
        for i, cat in enumerate(topic_data['categories']):
            add_category(topic_info, cat)

# Function to add new category input fields to a specific topic
def add_category(topic_info, category_data=None):
    index = len(topic_info['categories']) + 1  # Calculate the new category number
    new_category_input = widgets.Text(
        value=category_data if category_data else '',
        description=f'Category {index}:',
        disabled=False
    )
    remove_button = widgets.Button(description="Remove", layout=widgets.Layout(width='100px'))
    category_box = widgets.HBox([new_category_input, remove_button])

    remove_button.on_click(lambda btn: remove_category(topic_info, category_box))

    topic_info['categories'].append((new_category_input, category_box))
    update_categories_container(topic_info)

# Function to remove a specific category from a specific topic
def remove_category(topic_info, category_box):
    topic_info['categories'] = [(input_field, box) for input_field, box in topic_info['categories'] if box != category_box]
    update_categories_container(topic_info)

# Function to update the categories container for a specific topic
def update_categories_container(topic_info):
    topic_info['categories_container'].children = [box for _, box in topic_info['categories']]
    update_topics_container()

# Function to remove a specific topic
def remove_topic(topic_info):
    topics.remove(topic_info)
    update_topics_container()

# Function to update the topics container
def update_topics_container():
    topics_container.children = [td['topic_box'] for td in topics]         
            
            
# Save topics to a local file
def save_topics(b):
    saveFileName = save_container.children[0].value+".json"
    data = []
    for topic_info in topics:
        topic_data = {
            'topic_input': topic_info['topic_input'].value,
            'prompt': topic_info['prompt'].value,
            'categories': [category_input.value for category_input, _ in topic_info['categories']]
        }
        data.append(topic_data)
    json_data = json.dumps(data, indent=4)

    with open(saveFileName, 'w') as f:
        f.write(json_data)
    print("Saved topics to "+saveFileName)

    
    
# Load topics from a local file
def load_topics(b):
    loadFileName = load_container.children[0].value+".json"
    if os.path.exists(loadFileName):
        with open(loadFileName, 'r') as f:
            data = json.load(f)

        global topics
        topics = []
        topics_container.children = []

        for topic_data in data:
            add_topic(topic_data=topic_data)
        print("Loaded topics from "+loadFileName)
    else:
        print("No "+loadFileName+" file found")           
            
            
def load_topcis_by_Name(loadFileName):
    if os.path.exists(loadFileName):
        with open(loadFileName, 'r') as f:
            data = json.load(f)

        global topics
        topics = []

        for topic_data in data:
            add_topic(topic_data=topic_data)
        print("Loaded topics from "+loadFileName)
    else:
        print("No "+loadFileName+" file found")       
            
            
# Process all topics and categories on another button click
def show_topics_and_categories():
    for i, topic_info in enumerate(topics):
        print(f"Topic {i + 1}: {topic_info['topic_input'].value}")
        for j, (category_input, _) in enumerate(topic_info['categories']):
            print(f"  Category {j + 1}: {category_input.value}")           

def do_Classification_Button_Function():
    # Record the start time
    start_time = time.time()
    numberOfCorrectResults=[]
    if groupClassificationCheckbox.value: #Group Classification
        groupClassification(classify_CSV_box.children[0].value,with_Evaluation_Checkbox.value,numberOfCorrectResults,constrainedOutputCheckbox.value)
    else:                                 #SingleClassification
        singleClassification(freeText_container.children[0].value, 0.0, True, constrainedOutputCheckbox.value)
    end_time = time.time()
    elapsed_time = end_time - start_time
    saveName=classify_CSV_box.children[0].value+"_(result)"
    with open(saveName+".csv",'a',newline='',encoding='utf-8') as f:
        singleResult=[]
        singleResult.append(str(elapsed_time)+"seconds")
        writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerow(singleResult)   
        singleResult=[]
        singleResult.append("Model:")
        singleResult.append(model)
        writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerow(singleResult)   
        singleResult=[]
        singleResult.append("ModelType:")
        singleResult.append(modelType)        
        writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerow(singleResult)   
        if(with_Evaluation_Checkbox.value):
            singleResult=["",""]
            for number in numberOfCorrectResults:
                singleResult.append("")
                singleResult.append(number)
            writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
            writer.writerow(singleResult) 
        for l in range(len(selectOptions)):
            singleResult=["Prompt:"]
            singleResult.append(topics[l]['prompt'].value)
            singleResult.append("Topic:")
            singleResult.append(topics[l]['topic_input'].value)
            singleResult.append("Categories:")
            singleResult.append(selectOptions[l])
            writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
            writer.writerow(singleResult)
        
        
        singleResult=["Time:"]
        singleResult.append(str(elapsed_time)+"seconds")
        writer = csv.writer(f , delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerow(singleResult)   
    print(f"This classification took {elapsed_time:.2f} seconds")

        
        




In [ ]:

    
    

#GroupClassification Checkbox
groupClassificationCheckbox=widgets.Checkbox(
    value=False,
    description='Group Classification',
    disabled=False
)
display(groupClassificationCheckbox)


groupClassificationCheckbox.observe(on_value_change, names='value')

#ConstrainedOutput Checkbox
constrainedOutputCheckbox=widgets.Checkbox(
    value=True,
    description='Constrained Output',
    disabled=False
)
display(constrainedOutputCheckbox)



#FreeText Container
freeText_container = widgets.VBox()
display(freeText_container)

freeText_input = widgets.Text(
    value="",
    description='FreeText:',
    disabled=False
)

freeText_container.children = [freeText_input]



#Classify CSV Container
classify_CSV_input = widgets.Text(
        value="data",
        description='CSV File:',
        disabled=False
    )

with_Evaluation_Checkbox=widgets.Checkbox(
    value=False,
    description='Evaluation',
    disabled=False
)

classify_CSV_box = widgets.HBox([classify_CSV_input,with_Evaluation_Checkbox])
classify_CSV_box.layout.visibility = "hidden"

display(classify_CSV_box)





# Container to hold all topics
topics_container = widgets.VBox()
display(topics_container)




plus_topic_button = widgets.Button(description="Add Topic")
plus_topic_button.on_click(add_topic)
display(plus_topic_button)




save_container = widgets.HBox()
display(save_container)



saveFileText_input = widgets.Text(
    value="topics",
    description='File Name:',
    disabled=False
)


save_button = widgets.Button(description="Save Topics")
save_button.on_click(save_topics)
save_container.children = [saveFileText_input,save_button]





load_container = widgets.HBox()
display(load_container)



loadFileText_input = widgets.Text(
    value="topics",
    description='File Name:',
    disabled=False
)


load_button = widgets.Button(description="Load Topics")
load_button.on_click(load_topics)
load_container.children = [loadFileText_input,load_button]





show_all_button = widgets.Button(description="Show Topics and Categories")
show_all_button.on_click(lambda b: show_topics_and_categories())
display(show_all_button)




classify_button = widgets.Button(description="Do Classification")
classify_button.on_click(lambda b: do_Classification_Button_Function())
display(classify_button)


        
